# Spec Check Test

In [ ]:
!python --version

Python 3.7.13


In [ ]:
from psutil import *

In [ ]:
cpu_count() #Display Number of CPU's

2

In [ ]:
cpu_stats()

scpustats(ctx_switches=1741226, interrupts=1399664, soft_interrupts=1233136, syscalls=0)

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   39G   70G  36% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  812M  59% /sbin/docker-init
tmpfs           6.4G   40K  6.4G   1% /var/colab
/dev/sda1        81G   43G   39G  53% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  4.4M   15G   1% /content/drive


In [ ]:
virtual_memory()

svmem(total=13617745920, available=12753747968, percent=6.3, used=615706624, free=10705526784, active=1121636352, inactive=1587695616, buffers=119861248, cached=2176651264, shared=1236992, slab=146276352)

# Initializing



Importing Files from Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')   # To access gdrive files

Mounted at /content/drive


Download necessary libraries

In [ ]:
%%time
!pip install Sastrawi                         # Sastrawi for stemming Indonesia sentences

import nltk                                   # Natural Language Toolkit 
nltk.download('punkt')                        # NLTK word and sentence tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 8.9 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 1.15 s, sys: 340 ms, total: 1.49 s
Wall time: 7.75 s


Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import word_tokenize
from google.colab import files

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Initializing Dataset

In [ ]:
slang = pd.read_csv('/content/drive/MyDrive/Combined_Slang_Dictionary.csv')
dataset = pd.read_csv('/content/drive/MyDrive/NewTokopediaDataset - Sheet1.csv')

Dataset Information

In [ ]:
def check_dataset_information():
  print('Total reviews: ' + str(len(dataset)))
  print('Dataset Columns: ')
  for columns in  dataset.columns:
    print('- ' + columns)
  print('Number of NULL dataset: ')
  print(str(dataset.isnull().sum()))

  print()

  print('Total slang words: ' + str(len(slang)))
  print('Slang Columns: ')
  for columns in  slang.columns:
    print('- ' + columns)
  print('Number of NULL slang: ')
  print(str(slang.isnull().sum()))

In [ ]:
check_dataset_information()

Total reviews: 5400
Dataset Columns: 
- Category
- Product Name
- Seller
- Location
- Price
- Overall Rating
- Number Sold
- Total Reviews
- Link
- Customer Rating
- Review
- Emotion
- Sentiment
Number of NULL dataset: 
Category            0
Product Name       12
Seller             13
Location            0
Price               0
Overall Rating      0
Number Sold         0
Total Reviews       0
Link                0
Customer Rating     0
Review              0
Emotion             2
Sentiment           0
dtype: int64

Total slang words: 15729
Slang Columns: 
- Unnamed: 0
- Before
- After
Number of NULL slang: 
Unnamed: 0    0
Before        0
After         0
dtype: int64


Removing Unnecessary Columns

In [ ]:
useless_columns = ['Category', 'Product Name', 'Seller', 'Location', 'Price', 'Overall Rating', 'Number Sold', 'Total Reviews', 'Link', 'Customer Rating', 'Emotion']
dataset = dataset.drop(useless_columns, 1)

slang = slang.drop('Unnamed: 0', 1)

In [ ]:
check_dataset_information()

Total reviews: 5400
Dataset Columns: 
- Review
- Sentiment
Number of NULL dataset: 
Review       0
Sentiment    0
dtype: int64

Total slang words: 15729
Slang Columns: 
- Before
- After
Number of NULL slang: 
Before    0
After     0
dtype: int64


# Data Cleaning

Replace Slang Words function

In [ ]:
dataset.at[40, 'Review']

'Sangat kecewa. Baru 4 bulan scroll sudah rusak.'

In [ ]:
dataset.at[843, 'Review']

'Td nya sy request utk cancel mo ganti p’byran nya pake gopay dr ovo td nya krn ada cash back, tp nggak dicancel sm toko tetap diproses aja. Sgt m’ngecewakan.'

In [ ]:
dataset.at[4000, 'Review']

'packing tdk dikemas dan asal kirim lgsung pdhal kondisi dusnya sudah rusak. mgkin karena harga barangnya murah Rp.25.000. jd tdk dikemas rapih.'

In [ ]:
dataset.at[5321, 'Review']

'Bersih dari debu, sekali jalan, bagus... dan pengiriman ok sih rapi dlm box'

In [ ]:
dataset.at[316, 'Review']

'Proses kemas/kirim LAMA, seller ditanya hanya jawab MOHON SABAR, MOHON TUNGGU. Tidak kasih kepastian kapan kirim. Hrs nya dijawab BISA / TIDAK. End up barang di pickup sendiri ke gudang kurir krn perlu.'

In [ ]:
def slang_cleaning():
  for i, review in enumerate(dataset['Review']):
    temp = ""
    tokenized = word_tokenize(review)                               # Split the words

    for j, word in enumerate(tokenized):
      flag = word in slang['Before'].values                         # Check if the value is a slang word

      if flag == True:
        slang_index = slang[slang['Before'] == word].index.values   # Getting the index of the slang word
        word = slang.at[slang_index[0], 'After']                    # Replacing the slang word with the correct word
        
      temp = temp + word + " "                                      # Concatenate all the split words into a sentence
    
    dataset.at[i, 'Review'] = temp      

  new_sentence = str(dataset.at[843, 'Review'])                            

Cleaning

In [ ]:
%%time

original_sentence = str(dataset.at[843, 'Review'])
print('Original:                  ' + original_sentence)

# Lower case the reviews
dataset['Review'] = dataset['Review'].str.lower()                 
new_sentence = str(dataset.at[843, 'Review'])
print('Lower Cased:               ' + new_sentence)

# Replacing slang words
slang_cleaning()                                                  
new_sentence = str(dataset.at[843, 'Review'])
print('Replace Slang:             ' + new_sentence)

# Removing numbers
dataset['Review'] = dataset['Review'].str.replace(r'\d+','')      
new_sentence = str(dataset.at[843, 'Review'])
print('Removed Numbers:           ' + new_sentence)

# Removing symbols
dataset['Review'] = dataset['Review'].str.replace(r'[^\w\s]','')  
new_sentence = str(dataset.at[843, 'Review'])
print('Removed Symbols:           ' + new_sentence)

# Removing extra whitespace
dataset['Review'] = dataset['Review'].str.replace(r'\s{2,}',' ')  
new_sentence = str(dataset.at[843, 'Review'])
print('Removed Extra Whitespace:  ' + new_sentence)

Original:                  Td nya sy request utk cancel mo ganti p’byran nya pake gopay dr ovo td nya krn ada cash back, tp nggak dicancel sm toko tetap diproses aja. Sgt m’ngecewakan.
Lower Cased:               td nya sy request utk cancel mo ganti p’byran nya pake gopay dr ovo td nya krn ada cash back, tp nggak dicancel sm toko tetap diproses aja. sgt m’ngecewakan.
Replace Slang:             tadi dia saya request untuk cancel mau ganti p ’ byran dia pakai gopay dari  ovo tadi dia karena ada cash back , tapi tidak dicancel sama  toko tetap diproses saja . sangat sama ’ ngecewakan . 
Removed Numbers:           tadi dia saya request untuk cancel mau ganti p ’ byran dia pakai gopay dari  ovo tadi dia karena ada cash back , tapi tidak dicancel sama  toko tetap diproses saja . sangat sama ’ ngecewakan . 
Removed Symbols:           tadi dia saya request untuk cancel mau ganti p  byran dia pakai gopay dari  ovo tadi dia karena ada cash back  tapi tidak dicancel sama  toko tetap diproses saja

In [ ]:
chosen_one = [40, 843, 4000, 5321, 316]

In [ ]:
for i in range(5):
  print(dataset.at[chosen_one[i], 'Review'])

sangat kecewa baru bulan scroll sudah rusak 
tadi dia saya request untuk cancel mau ganti p byran dia pakai gopay dari ovo tadi dia karena ada cash back tapi tidak dicancel sama toko tetap diproses saja sangat sama ngecewakan 
packing tidak dikemas dan asal kirim langsung padahal kondisi dusnya sudah rusak mungkin karena harga barangnya murah rp jadi tidak dikemas rapi 
bersih dari debu sekali jalan bagus dan pengiriman ok sih rapi dalam box 
proses kemaskirim lama seller ditanya hanya jawab mohon sabar mohon tunggu tidak kasih kepastian kapan kirim harus dia dijawab bisa tidak end up barang di pickup sendiri ke gudang kurir karena perlu 


In [ ]:
check_dataset_information()

Total reviews: 5400
Dataset Columns: 
- Review
- Sentiment
Number of NULL dataset: 
Review       0
Sentiment    0
dtype: int64

Total slang words: 15729
Slang Columns: 
- Before
- After
Number of NULL slang: 
Before    0
After     0
dtype: int64


# Preprocessing

Initialize Preprocessing Variables

In [ ]:
stopwords_id = pd.read_csv('/content/drive/MyDrive/stopwords-id.txt',delimiter='\n')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

Stopword + Stemming

In [ ]:
%%time
for i, sentence in enumerate(dataset['Review']):
  temp = ""
  tokenized = word_tokenize(sentence)               # Split the sentence

  for j, word in enumerate(tokenized):
    if word in stopwords_id['Stopwords'].values:    # Removing stop-words
      continue
    else:
        temp = temp + word + " "

  if temp == "":
    temp = sentence
  
  dataset.at[i,'Review'] = stemmer.stem(temp)       # Stemming the sentence

final_sentence = str(dataset.at[843, 'Review'])

CPU times: user 12min 31s, sys: 2.67 s, total: 12min 33s
Wall time: 12min 36s


In [ ]:
for i in range(5):
  print(dataset.at[chosen_one[i], 'Review'])

kecewa scroll rusak
request cancel ganti p byran pakai gopay ovo cash back dicancel toko proses ngecewakan
packing kemas kirim dus rusak harga barang murah rp kemas rapi
bersih debu bagus kirim sih rapi box
kemaskirim seller mohon sabar mohon tunggu kasih pasti kirim end up pickup gudang kurir


In [ ]:
print('Before preprocessing:  ' + new_sentence)
print('After preprocessing:   ' + final_sentence)

Before preprocessing:  tadi dia saya request untuk cancel mau ganti p byran dia pakai gopay dari ovo tadi dia karena ada cash back tapi tidak dicancel sama toko tetap diproses saja sangat sama ngecewakan 
After preprocessing:   request cancel ganti p byran pakai gopay ovo cash back dicancel toko proses ngecewakan


In [ ]:
check_dataset_information()

Total reviews: 5400
Dataset Columns: 
- Review
- Sentiment
Number of NULL dataset: 
Review       0
Sentiment    0
dtype: int64

Total slang words: 15729
Slang Columns: 
- Before
- After
Number of NULL slang: 
Before    0
After     0
dtype: int64


# Downloading the Cleaned Dataset

In [ ]:
dataset.to_csv(r'TokopediaCleanedStemmedFIXED.csv')
files.download('TokopediaCleanedStemmedFIXED.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>